# Evaluate Model with Amazon SageMaker Processing Jobs and Scikit-Learn

Often, distributed data processing frameworks such as Scikit-Learn are used to pre-process data sets in order to prepare them for training.  

In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Scikit-Learn in a managed SageMaker environment to run our processing workload.

# NOTE:  THIS NOTEBOOK WILL TAKE A 5-10 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.

![](img/prepare_dataset_bert.png)

![](img/processing.jpg)

## Contents

1. Setup Environment
1. Setup Input Data
1. Setup Output Data
1. Build a Spark container for running the processing job
1. Run the Processing Job using Amazon SageMaker
1. Inspect the Processed Output Data

# Setup Environment

Let's start by specifying:
* The S3 bucket and prefixes that you use for training and model data. Use the default bucket specified by the Amazon SageMaker session.
* The IAM role ARN used to give processing and training access to the dataset.

In [62]:
import sagemaker
import boto3

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [63]:
%store -r training_job_name

In [64]:
try:
    training_job_name
    print("[OK]")
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the previous TRAIN section before you continue.")
    print("+++++++++++++++++++++++++++++++")

[OK]


In [65]:
print(training_job_name)

tensorflow-training-2025-02-16-04-30-21-406


In [66]:
raw_input_data_s3_uri = "s3://{}/amazon-reviews-pds/tsv/".format(bucket)
%store raw_input_data_s3_uri

Stored 'raw_input_data_s3_uri' (str)


In [67]:
try:
    raw_input_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [68]:
print(raw_input_data_s3_uri)

s3://sagemaker-us-east-1-421477113665/amazon-reviews-pds/tsv/


In [69]:
max_seq_length=64
%store max_seq_length

Stored 'max_seq_length' (int)


In [70]:
try:
    max_seq_length
    print("[OK]")
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the previous TRAIN section before you continue.")
    print("+++++++++++++++++++++++++++++++")

[OK]


In [71]:
print(max_seq_length)

64


In [72]:
experiment_name = "Amazon-Customer-Reviews-BERT-Experiment-1739670178"
%store experiment_name

Stored 'experiment_name' (str)


In [73]:
try:
    experiment_name
    print("[OK]")
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the previous TRAIN section before you continue.")
    print("+++++++++++++++++++++++++++++++")

[OK]


In [74]:
print(experiment_name)

Amazon-Customer-Reviews-BERT-Experiment-1739670178


In [75]:
trial_name = "trial-1739670178"
%store trial_name

Stored 'trial_name' (str)


In [76]:
try:
    trial_name
    print("[OK]")
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the previous TRAIN section before you continue.")
    print("+++++++++++++++++++++++++++++++")

[OK]


In [77]:
print(trial_name)

trial-1739670178


In [78]:
print(training_job_name)

tensorflow-training-2025-02-16-04-30-21-406


In [79]:
from sagemaker.tensorflow.estimator import TensorFlow

describe_training_job_response = sm.describe_training_job(TrainingJobName=training_job_name)
print(describe_training_job_response)

{'TrainingJobName': 'tensorflow-training-2025-02-16-04-30-21-406', 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:421477113665:training-job/tensorflow-training-2025-02-16-04-30-21-406', 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-421477113665/tensorflow-training-2025-02-16-04-30-21-406/output/model.tar.gz'}, 'TrainingJobStatus': 'Completed', 'SecondaryStatus': 'Completed', 'HyperParameters': {'enable_checkpointing': 'false', 'enable_sagemaker_debugger': 'true', 'enable_tensorboard': 'true', 'epochs': '2', 'epsilon': '1e-06', 'freeze_bert_layer': 'false', 'learning_rate': '0.001', 'max_seq_length': '64', 'model_dir': '"s3://sagemaker-us-east-1-421477113665/tensorflow-training-2025-02-16-04-30-21-406/model"', 'run_sample_predictions': 'true', 'run_test': 'true', 'run_validation': 'true', 'sagemaker_container_log_level': '20', 'sagemaker_job_name': '"tensorflow-training-2025-02-16-04-30-21-406"', 'sagemaker_program': '"tf_bert_reviews.py"', 'sagemaker_region': '"us-ea

In [80]:
model_dir_s3_uri = describe_training_job_response["ModelArtifacts"]["S3ModelArtifacts"].replace("model.tar.gz", "")
model_dir_s3_uri

's3://sagemaker-us-east-1-421477113665/tensorflow-training-2025-02-16-04-30-21-406/output/'

# Run the Processing Job using Amazon SageMaker

Next, use the Amazon SageMaker Python SDK to submit a processing job using our custom python script.

# Create the `Experiment Config`

In [81]:
experiment_config = {
    "ExperimentName": experiment_name,
    "TrialName": trial_name,
    "TrialComponentDisplayName": "evaluate",
}

# Set the Processing Job Hyper-Parameters 

In [82]:
processing_instance_type = "ml.m5.xlarge"
processing_instance_count = 1

# Choosing a `max_seq_length` for BERT
Since a smaller `max_seq_length` leads to faster training and lower resource utilization, we want to find the smallest review length that captures `80%` of our reviews.

Remember our distribution of review lengths from a previous section?

```
mean         51.683405
std         107.030844
min           1.000000
10%           2.000000
20%           7.000000
30%          19.000000
40%          22.000000
50%          26.000000
60%          32.000000
70%          43.000000
80%          63.000000
90%         110.000000
100%       5347.000000
max        5347.000000
```

![](img/review_word_count_distribution.png)

Review length `63` represents the `80th` percentile for this dataset.  However, it's best to stick with powers-of-2 when using BERT.  So let's choose `64` as this is the smallest power-of-2 greater than `63`.  Reviews with length > `64` will be truncated to `64`.

In [83]:
from sagemaker.sklearn.processing import SKLearnProcessor

processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    max_runtime_in_seconds=7200,
)

[02/16/25 04:54:58] INFO     Defaulting to only available Python version: py3                     ]8;id=983158;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=963284;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#610\610]8;;\

In [84]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

processor.run(
    code="evaluate_model_metrics.py",
    inputs=[
        ProcessingInput(
            input_name="model-tar-s3-uri", source=model_dir_s3_uri, destination="/opt/ml/processing/input/model/"
        ),
        ProcessingInput(
            input_name="evaluation-data-s3-uri",
            source=raw_input_data_s3_uri,
            destination="/opt/ml/processing/input/data/",
        ),
    ],
    outputs=[
        ProcessingOutput(s3_upload_mode="EndOfJob", output_name="metrics", source="/opt/ml/processing/output/metrics"),
    ],
    arguments=["--max-seq-length", str(max_seq_length)],
    #experiment_config=experiment_config,
    logs=True,
    wait=False,
)

[02/16/25 04:54:59] INFO     Creating processing-job with name                                      ]8;id=446850;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=336479;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1575\1575]8;;\
                             sagemaker-scikit-learn-2025-02-16-04-54-58-963                                        

In [85]:
scikit_processing_job_name = processor.jobs[-1].describe()["ProcessingJobName"]
print(scikit_processing_job_name)

sagemaker-scikit-learn-2025-02-16-04-54-58-963


In [86]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, scikit_processing_job_name
        )
    )
)

/tmp/ipykernel_9317/1926661146.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [87]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, scikit_processing_job_name
        )
    )
)

/tmp/ipykernel_9317/267036670.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [88]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, scikit_processing_job_name, region
        )
    )
)

/tmp/ipykernel_9317/2163845711.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Monitor the Processing Job

In [89]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=scikit_processing_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

print(processing_job_description)

{'ProcessingInputs': [{'InputName': 'model-tar-s3-uri', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-421477113665/tensorflow-training-2025-02-16-04-30-21-406/output/', 'LocalPath': '/opt/ml/processing/input/model/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'evaluation-data-s3-uri', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-421477113665/amazon-reviews-pds/tsv/', 'LocalPath': '/opt/ml/processing/input/data/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-421477113665/sagemaker-scikit-learn-2025-02-16-04-54-58-963/input/code/evaluate_model_metrics.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyR

In [90]:
processing_evaluation_metrics_job_name = processing_job_description["ProcessingJobName"]
print(processing_evaluation_metrics_job_name)

sagemaker-scikit-learn-2025-02-16-04-54-58-963


In [91]:
%%time

running_processor.wait(logs=False)

...................................................................................................................................................................................!CPU times: user 626 ms, sys: 55.6 ms, total: 682 ms
Wall time: 15min 7s


# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

# Inspect the Processed Output Data

Take a look at a few rows of the transformed dataset to make sure the processing was successful.

In [92]:
processing_job_description = running_processor.describe()

output_config = processing_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    if output["OutputName"] == "metrics":
        processed_metrics_s3_uri = output["S3Output"]["S3Uri"]

print(processed_metrics_s3_uri)

s3://sagemaker-us-east-1-421477113665/sagemaker-scikit-learn-2025-02-16-04-54-58-963/output/metrics


In [93]:
!aws s3 ls $processed_metrics_s3_uri/

2025-02-16 05:08:44      20455 confusion_matrix.png
2025-02-16 05:08:44         41 evaluation.json


## Show the test accuracy

In [94]:
import json
from pprint import pprint

evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(processed_metrics_s3_uri))

pprint(json.loads(evaluation_json))

{'metrics': {'accuracy': {'value': 0.1}}}


In [95]:
!aws s3 cp $processed_metrics_s3_uri/confusion_matrix.png ./model_evaluation/

import time

time.sleep(10)  # Slight delay for our notebook to recognize the newly-downloaded file

download: s3://sagemaker-us-east-1-421477113665/sagemaker-scikit-learn-2025-02-16-04-54-58-963/output/metrics/confusion_matrix.png to model_evaluation/confusion_matrix.png


In [96]:
%%html

<img src='./model_evaluation/confusion_matrix.png'>

# Pass Variables to the Next Notebook(s)

In [97]:
%store processing_evaluation_metrics_job_name

Stored 'processing_evaluation_metrics_job_name' (str)


In [98]:
%store processed_metrics_s3_uri

Stored 'processed_metrics_s3_uri' (str)


In [99]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                      -> 'automl-dm-15-21-49-33'
autopilot_endpoint_arn                                -> 'arn:aws:sagemaker:us-east-1:421477113665:endpoint
autopilot_endpoint_name                               -> 'automl-dm-ep-15-23-48-30'
autopilot_model_arn                                   -> 'arn:aws:sagemaker:us-east-1:421477113665:processi
autopilot_model_name                                  -> 'automl-dm-model-15-22-32-37'
autopilot_train_s3_uri                                -> 's3://sagemaker-us-east-1-421477113665/data/amazon
balanced_bias_data_jsonlines_s3_uri                   -> 's3://sagemaker-us-east-1-421477113665/bias-detect
balanced_bias_data_s3_uri                             -> 's3://sagemaker-us-east-1-421477113665/bias-detect
bias_data_s3_uri                                      -> 's3://sagemaker-us-east-1-421477113665/bias-detect
experiment_name                                       -> 'A

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}